<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="cognitiveclass.ai logo">
</center>


# Singular-Value Decomposition (SVD)

Estimated time needed: **45** minutes

You are a computer vision engineer and a self-driving car company has hired you.  The company would like you to remove all objects other than the street from several security videos using singular value decomposition to help test their vision system. You will use the  <a href="www.svcl.ucsd.edu/projects/background_subtraction/ucsdbgsub_dataset.html"> Background Subtraction in Dynamic dataset  </a> from the  Statistical Visual Computing Laboratory (SVCL) at UCSD.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/images/15104006386_1bf6bfe96a_b.jpeg" width="600">



## Table of Contents

<ol>
    <li><a href="#Objectives">Objectives</a></li>
    <li>
        <a href="#Setup">Setup</a>
        <ol>
            <li><a href="#Installing-Required-Libraries">Installing Required Libraries</a></li>
            <li><a href="#Importing-Required-Libraries">Importing Required Libraries</a></li>
            <li><a href="#Defining-Helper-Functions">Defining Helper Functions</a></li>
        </ol>
    </li>
    <li><a href="#Singular Value Decomposition">Singular Value Decomposition</a></li>
    <li>
        <a href="#Truncated SVD">Truncated SVD</a>
        <ol>
            <li><a href="#Truncated SVD in Sklearn">Truncated SVD in Sklearn</a></li>
            <li><a href="#Background Model using SVD">Background Model using SVD</a></li>
        </ol>
    </li>
    <li>
        <a href="#Exercises">Exercises</a>
        <ol>
            <li><a href="#Exercise 1">Exercise 1</a></li>
            <li><a href="#Exercise 2">Exercise 2</a></li>
            <li><a href="#Exercise 3">Exercise 3</a></li>
            <li><a href="#Exercise 4">Exercise 4</a></li>
            <li><a href="#Exercise 5">Exercise 5</a></li>
        </ol>
    </li>
    <li><a href="#SVD from Scratch (optional)">SVD from Scratch (optional)</a></li>
    <li><a href="#Relationship between SVD and PCA (optional)">Relationship between SVD and PCA (optional)</a></li>
    
 </ol>    
     
  


## Objectives


*   **Understand** what is SVD in terms of Matrix Decomposition
*   **Understand** Truncated SVD 
*   **Apply** Truncated SVD  using numpy and Sklearn
*   **Apply** Truncated SVD  to real data 
*   **Understand** the relationship between SVD and PCA (optional)


In [2]:
import skillsnetwork

await skillsnetwork.prepare("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/data/traffic.tar.gz", overwrite=True)
await skillsnetwork.prepare("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/data/peds.tar.gz", overwrite=True)
await skillsnetwork.prepare("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/data/boats.tar.gz", overwrite=True)


  0%|          | 0/382 [00:00<?, ?it/s]

Saved to '.'


  0%|          | 0/341 [00:00<?, ?it/s]

Saved to '.'


  0%|          | 0/64 [00:00<?, ?it/s]

Saved to '.'


## Setup


### Installing  required libraries


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from os import listdir, getcwd
from os.path import isfile, join
from random import randint
from PIL import Image
from sympy import Matrix, init_printing, Symbol
from numpy.linalg import qr, eig, inv, matrix_rank, inv, svd

In [5]:
init_printing()

### Defining Helper functions


In [6]:
def get_data_Matrix (mypath="peds"):
    cwd = getcwd()

    mypath = join(cwd,mypath)
    files = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f)) and f.startswith(".") == False]
    # Read image
    img = Image.open(files[0])
    I = np.array(img)
    # Output Images

    Length, Width = I.shape
   
    X = np.zeros((len(files), Length * Width))
    for i, file in enumerate(files):
        img = Image.open(file)
        I = np.array(img)
        X[i, :] = I.reshape(1, -1)
    return X, Length, Width

## Singular Value Decomposition
The Singular-Value Decomposition, or SVD for short, will decompose a real or complex $N × D$ matrix $\mathbf{X}$ of rank $r$ as follows:


$$\mathbf{X}= \mathbf {US V^{T}}$$


In many applications  $N \ge D$, but SVD can be used for any matrix $\mathbf{X}$. For example, in computer vision and image processing tasks we sometimes have $D \ge N$.


The matrix $\mathbf{S}$ contains the nonnegative <b>singular values</b> of $\mathbf{X}$, with diagonal entries ${\sigma _{i}}$ else $0$, the entries are ordered by importance in descending order with respect to $i$, i.e:

$$\sigma _{1}>\sigma _{2},..>\sigma _{r}$$


The matrix $\mathbf {U}$ is $NxD$ and  has the orthornormal columns  $\mathbf{u}_{1}, ..., \mathbf{u}_{D} $ called the <b>left singular vectors</b>.

The matrix $\mathbf {V}$ is $DxD$ and has the orthornormal columns $\mathbf{v}_{1}, ..., \mathbf{v}_{D}$ called the <b>right singular vectors</b>  (note  that $\mathbf{V}$ transpose, $\mathbf {V^{T}}$ is returned as output in numpy's `svd` function).


 SVD decomposition returns the full shape of a non-square matrix, the non colored  parts of the decomposition **N-D** terms in the matrix U are zeros, we see many of the squares are redundant.


<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/images/matrix.png" width="600" alt="full svd">


Consider the matrix $\mathbf{X}$:


In [7]:
X = np.array([[1.0, 2], [2, 1], [3, 3]])
Matrix(X)

⎡1.0  2.0⎤
⎢        ⎥
⎢2.0  1.0⎥
⎢        ⎥
⎣3.0  3.0⎦

We can perform SVD on any matrix in numpy by using the function `svd` from `numpy.linalg`:


In [8]:
U, s, VT = svd(X, full_matrices=False)

When $\mathbf{X}$ is a 2D array, it is factorized as $U\times np.diag(s)\times V^T$, where $U$ and $V^T$ are 2D orthogonal  matrices  and $s$ is an 1D array of $\mathbf{X}$'s singular values. When $\mathbf{X}$ is higher-dimensional (such as sparse matrices), the parameter ```full_matrices=False```, the skinny SVD is used and the zero elements are dropped. This can be summarized in the following image:


<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/images/skinny-SVD.png" width="600" alt="skinnysvd">


We have the <b>left singular vectors</b> of $\mathbf{X}$:


In [9]:
Matrix(U)

⎡-0.408248290463863    0.707106781186548  ⎤
⎢                                         ⎥
⎢-0.408248290463863   -0.707106781186547  ⎥
⎢                                         ⎥
⎣-0.816496580927726  -1.57698996923239e-16⎦

We have the <b>singular values</b> of $\mathbf{X}$, as the output is an 1-D array we use the function ```np.diag``` to convert the output into a diagonal matrix:


In [11]:
Matrix(s)

⎡5.19615242270663⎤
⎢                ⎥
⎣      1.0       ⎦

In [10]:
S = np.diag(s)
Matrix(S)

⎡5.19615242270663  0.0⎤
⎢                     ⎥
⎣      0.0         1.0⎦

Finally we have the <b>right singular vectors</b> of $\mathbf{X}$, as the output is transposed they are the rows of matrix ```VT```:


In [12]:
Matrix(VT)

⎡-0.707106781186547  -0.707106781186547⎤
⎢                                      ⎥
⎣-0.707106781186547  0.707106781186547 ⎦